In [ ]:
import sys
sys.path.append("modules")

In [ ]:
import os, glob
from PIL import Image

import numpy as np
import scipy.interpolate as interp
import scipy.fft as fft
from scipy.stats import linregress

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import modules.time_to_H as time_to_H
import modules.compute_flexion as flex
import modules.parameters as setup

import modules.smooth_PIV as smooth

In [ ]:
from importlib import reload
reload(smooth)

In [ ]:
# Read data

pathExp = "../data/Experiments/h_1p5cm/tauC_100Pa/1_CCGM_500um_375nm_45uN_100Pa_h_1p5cm_m_137g"

pathFields = os.path.join(pathExp, "muDIC_fromRaw")
pathSnapshots = os.path.join(pathExp, "snapshots_preProcessed")

h_layer = 1.5e-2
dt_camera = 0.1

# For cropped data
xMin_crop, xMax_crop = 0, 1772
yMin_crop, yMax_crop = 0, 1165

frames_true = np.load(os.path.join(pathFields, "frames.npy")).astype(int)
coords = np.load(os.path.join(pathFields, "coords.npy"))
trueStrain_raw = np.load(os.path.join(pathFields, "true_strain.npy"))
trueStrainXX_raw = trueStrain_raw[0,0,0]

times_true = dt_camera * frames_true
heights = time_to_H.time_to_H(times_true, Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax)

with open(os.path.join(pathFields, 'params.txt')) as file:
    paramsDIC = file.read()

# Undo the change of origin due to cropping
coords[0,0] += xMin_crop
coords[0,1] += yMin_crop

# Correct errors nbFrames
frames_true = frames_true[:trueStrainXX_raw.shape[-1]]
times_true = times_true[:trueStrainXX_raw.shape[-1]]
heights = heights[:trueStrainXX_raw.shape[-1]]

In [ ]:
# Read calibration

pathCalibration = "../data/Calibrations/Top_grey_02_08/muDIC"

dt_camera_calib = 0.1

# For cropped data
xMin_crop_calib, xMax_crop_calib = 0, 2448
yMin_crop_calib, yMax_crop_calib = 0, 2048

frames_true_calib = np.load(os.path.join(pathCalibration, "frames.npy"))
coords_calib = np.load(os.path.join(pathCalibration, "coords.npy"))
trueStrain_calib = np.load(os.path.join(pathCalibration, "true_strain.npy"))

times_true_calib = dt_camera_calib * frames_true_calib
heights_calib = time_to_H.time_to_H(times_true_calib, Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax)

# Undo the change of origin due to cropping
coords_calib[0,0] += xMin_crop_calib
coords_calib[0,1] += yMin_crop_calib

# Keep only the rising part
duration_rising = time_to_H.compute_time_rising(Hmax=setup.Hmax, vMax=setup.vmax, aMax=setup.amax)
mask_rising_calib = (times_true_calib <= duration_rising)

frames_true_calib = frames_true_calib[mask_rising_calib]
coords_calib = coords_calib[:,:,:,:,mask_rising_calib]
trueStrain_calib = trueStrain_calib[:,:,:,:,:,mask_rising_calib]
times_true_calib = times_true_calib[mask_rising_calib]
heights_calib = heights_calib[mask_rising_calib]

In [ ]:
# Test position

xExtend = coords[0,0,:,:,0].min(), coords[0,0,:,:,0].max()
yExtend = coords[0,1,:,:,0].min(), coords[0,1,:,:,0].max()

xExtend_calib = coords_calib[0,0,:,:,0].min(), coords_calib[0,0,:,:,0].max()
yExtend_calib = coords_calib[0,1,:,:,0].min(), coords_calib[0,1,:,:,0].max()

fig, ax = plt.subplots()

ax.plot((xExtend[0], xExtend[1], xExtend[1], xExtend[0]), (yExtend[0], yExtend[0], yExtend[1], yExtend[1]), 'bo')
ax.plot((xExtend_calib[0], xExtend_calib[1], xExtend_calib[1], xExtend_calib[0]), 
        (yExtend_calib[0], yExtend_calib[0], yExtend_calib[1], yExtend_calib[1]), 'ko')

plt.show()

In [ ]:
# Print shapes

print("From experiment")
print(f"Frames: {frames_true.shape}")
print(f"Coordinates/Strain: {coords[0,0].shape}")

print("")

print("From calibration")
print(f"Frames: {frames_true_calib.shape}")
print(f"Coordinates/Strain: {coords_calib[0,0].shape}")

In [ ]:
# Prepare snapshots prefix

snapshot_names = glob.glob(pathSnapshots + "/*.tiff")

snapshot_prefix = snapshot_names[0].rsplit("_")
snapshot_prefix = "_".join(snapshot_prefix[:-1])

In [ ]:
## Fit center curcature with parabola

plotFit = True

x0 = coords[0,0,:,:,0]
x0_calib = coords_calib[0,0,:,:,0]

# For experiments
trueStrainXX_raw_avg_Y = trueStrainXX_raw.mean(axis=1)
x0_arr = x0[:,x0.shape[1]//2]

poly_coeff = np.polyfit(x0_arr, trueStrainXX_raw_avg_Y, deg=2)
x_maxStrain = np.where(np.abs(poly_coeff[0])>1e-15, -poly_coeff[1] / (2 * poly_coeff[0]), 0)

# For calibration
trueStrain_calib_xx = trueStrain_calib[0,0,0,:,:,:].mean(axis=1)
x0_arr_calib = x0_calib[:,x0_calib.shape[1]//2]

poly_coeff_calib = np.polyfit(x0_arr_calib, trueStrain_calib_xx, deg=2)
x_maxStrain_calib = np.where(np.abs(poly_coeff_calib[0])>1e-15, -poly_coeff_calib[1] / (2 * poly_coeff_calib[0]), 0)


if plotFit:
    frameIndex_toPlot = 90
    fig, ax = plt.subplots(ncols=2, figsize=(10,5))

    ax[0].plot(x0_arr, trueStrainXX_raw_avg_Y[:,frameIndex_toPlot], label="Measure")
    ax[0].plot(x0_arr, np.poly1d(poly_coeff[:,frameIndex_toPlot])(x0_arr), 'k:', label="Fit")
    ax[0].axvline(x_maxStrain[frameIndex_toPlot], ls="--", c='k')

    ax[0].legend()
    ax[0].set_title(f"Frame index {frameIndex_toPlot}")

    ax[1].plot(x_maxStrain)
    ax[1].axhline(x0_arr.mean(), c='k')
    ax[1].set_xlabel('Frame index')
    ax[1].set_ylabel('X_maxStrain')

    plt.show()

In [ ]:
# Create interpolator calibration (x0,t)->eps_t

# Average calibration along y
trueStrain_calib_xx = trueStrain_calib[0,0,0,:,:,:].mean(axis=1)
x0_calib = coords_calib[0,0,:,:,0].mean(axis=1)

# Create interpolator
trueStrain_calib_interp_fromX0 = interp.RegularGridInterpolator((x0_calib, times_true_calib), 
                                                         trueStrain_calib_xx, 
                                                         bounds_error=False, fill_value=None)


In [ ]:
# Create interpolator calibration (x(t),h)->eps_t

# Average calibration along y
trueStrain_calib_xx = trueStrain_calib[0,0,0,:,:,:].mean(axis=1) # shape (x,time)
x_calib = coords_calib[0,0,:,:,:].mean(axis=1)                   # shape (x,time)
heights_calib_2d = np.array([heights_calib for _ in range(x_calib.shape[0])])

# Create interpolator
trueStrain_calib_interp_fromXt = interp.LinearNDInterpolator(list(zip(x_calib.flatten(), heights_calib_2d.flatten())), 
                                                      trueStrain_calib_xx.flatten(),
                                                      fill_value=np.nan, rescale=True)

# Prepare the data (using calibration)

In [ ]:
# Correct the strain

x0 = coords[0,0,:,:,0]
y0 = coords[0,1,:,:,0]

trueStrainXX_correction = np.empty_like(trueStrain_raw[0,0,0])
curvatures = np.empty((2, times_true.size))

for i_time in range(times_true.size):

    time = times_true[i_time]
    height = heights[i_time]

    # Compute correction from (x,t)
    trueStrainXX_correction[:,:,i_time] = trueStrain_calib_interp_fromXt((coords[0,0,:,:,i_time].flatten(), 
                                                                          np.full(x0.size, height))).reshape(x0.shape)
    
    # Compute correction from (X0,t)
    #trueStrainXX_correction[:,:,i_time] = trueStrain_calib_interp_fromX0((x0.flatten(), 
    #                                                                      np.full(x0.size, time))).reshape(x0.shape)
    
    guess_alpha = [0.1, 0.01]
    curvatures[:,i_time] = flex.solve_curvature(setup.W, heights[i_time], setup.a, guess_alpha)

trueStrainXX_corrected = trueStrainXX_raw - trueStrainXX_correction
engStrainXX_corrected = np.exp(trueStrainXX_corrected) - 1

# Compare results with and without smoothing

In [ ]:
# Compute smoothed data

trueStrainXX_smoothed = np.empty_like(trueStrainXX_corrected)
smooth_factor = 1

for frame in range(trueStrainXX_smoothed.shape[-1]):
    trueStrainXX_smoothed[:,:,frame] = smooth.smooth_with_s(trueStrainXX_corrected[:,:,frame], smooth_factor)

In [ ]:
# Plot the strain along given lines (no smoothing)

frame_index = 40
offset_H = 0e-3

y_pix_list = [600, 700, 800, 900, 1000]
#y_pix_list = [1400, 1500, 1600, 1700]

print(f"Min/Max y-coord: {y0.min(), y0.max()}")

# Find nearest element for each y_pix
y_elem_list = []
for y_pix in y_pix_list:
    y_elem = np.argmin(np.abs(y_pix - y0[y0.shape[0]//2,:]))
    y_elem_list.append(y_elem)

# Plot
fig, ax = plt.subplots(figsize=(10, 5), ncols=2, tight_layout=True)

for y_pix, y_elem in zip(y_pix_list, y_elem_list):
    ax[0].plot(x0[:,y_elem], trueStrainXX_raw[:,y_elem,frame_index], label=f'y_pix = {y_pix}')
    ax[1].plot(x0[:,y_elem], trueStrainXX_corrected[:,y_elem,frame_index], label=f'y_pix = {y_pix}')

# Plot the y-averaged
ax[0].plot(x0[:,y_elem], trueStrainXX_raw[:,:,frame_index].mean(axis=1), label=f'y-averaged', c='k')
ax[1].plot(x0[:,y_elem], trueStrainXX_corrected[:,:,frame_index].mean(axis=1), label=f'y-averaged', c='k')


# Plot the theoretical strain due to flexion
kappa_min, kappa_max = flex.solve_curvature(setup.W, heights[frame_index]-offset_H, setup.a)
ax[1].axhline(np.log(1 + kappa_max * h_layer), ls=':', c='k', label="$\log \\left( 1 + \\kappa h \\right)$")

for axi in ax:
    axi.set_xlabel('Horizontal position $X_0$ [pix]')
    axi.legend(loc='best', ncol=2)

ax[0].set_ylabel("True strain $\epsilon^t_{xx}$ [-]")
ax[1].set_ylabel("True strain $\epsilon^t_{xx} - \epsilon^t_{xx, calib}$ [-]")

ax[0].set_title(f"True strain [H = {heights[frame_index] * 1e3:.1f}mm]")
ax[1].set_title(f"True strain (corrected) [H = {heights[frame_index] * 1e3:.1f}mm]")

ax[1].set_ybound([0, 0.225])

plt.show()

In [ ]:
# Plot the strain along given lines (smoothing)

frame_index = 40
offset_H = 0e-3

y_pix_list = [600, 700, 800, 900, 1000]
#y_pix_list = [1400, 1500, 1600, 1700]

print(f"Min/Max y-coord: {y0.min(), y0.max()}")

# Find nearest element for each y_pix
y_elem_list = []
for y_pix in y_pix_list:
    y_elem = np.argmin(np.abs(y_pix - y0[y0.shape[0]//2,:]))
    y_elem_list.append(y_elem)

# Plot
fig, ax = plt.subplots(figsize=(10, 5), ncols=2, tight_layout=True)

for y_pix, y_elem in zip(y_pix_list, y_elem_list):
    ax[0].plot(x0[:,y_elem], trueStrainXX_raw[:,y_elem,frame_index], label=f'y_pix = {y_pix}')
    ax[1].plot(x0[:,y_elem], trueStrainXX_smoothed[:,y_elem,frame_index], label=f'y_pix = {y_pix}')

# Plot the y-averaged
ax[0].plot(x0[:,y_elem], trueStrainXX_raw[:,:,frame_index].mean(axis=1), label=f'y-averaged', c='k')
ax[1].plot(x0[:,y_elem], trueStrainXX_smoothed[:,:,frame_index].mean(axis=1), label=f'y-averaged', c='k')


# Plot the theoretical strain due to flexion
kappa_min, kappa_max = flex.solve_curvature(setup.W, heights[frame_index]-offset_H, setup.a)
ax[1].axhline(np.log(1 + kappa_max * h_layer), ls=':', c='k', label="$\log \\left( 1 + \\kappa h \\right)$")

for axi in ax:
    axi.set_xlabel('Horizontal position $X_0$ [pix]')
    axi.legend(loc='best', ncol=2)

ax[0].set_ylabel("True strain $\epsilon^t_{xx}$ [-]")
ax[1].set_ylabel("True strain $\epsilon^t_{xx} - \epsilon^t_{xx, calib}$ [-]")

ax[0].set_title(f"True strain [H = {heights[frame_index] * 1e3:.1f}mm]")
ax[1].set_title(f"True strain (corrected) [H = {heights[frame_index] * 1e3:.1f}mm]")

ax[1].set_ybound([0, 0.225])

plt.show()